In [1]:
from utils.opt_utils import *
import h5py
import pickle
from qutip import *

In [2]:
config_dict = pc.config_dict

In [3]:
# Defining relevant values
q_ch = config_dict["qubit"]["ch"] 
A_ch = config_dict["storage_A"]["ch"]
B_ch = config_dict["storage_B"]["ch"]

wq = config_dict["qubit"]["freqs"]["g"]["ge"]
wa_A = config_dict["storage_A"]["freq"]
wa_B = config_dict["storage_B"]["freq"]

In [4]:
# ========== OPTIONS ========== #
max_segs = 50
us = 0.000001
time_start = 0 * us
time_stop = 1.2 * us
init_qamp = 6000
init_camp = 32000

num_drives = 2
chs = [2, 4]
init_freqs = [wq, wa_A]
verbose = True
plot_opt_pulses = False
plot_pulse_viewer = False
start_split_num = 200
n_seg_jump = 1

load_pulse_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\exp_1g_EXAMPLE'
load_times_file = r'opt_SNAP_times_1g.txt'
load_amps_file = r'opt_SNAP_amps_1g.txt'
use_loaded_data = False

save_dir = r'C:\Users\Wang_Lab\Documents\GitLab\quantum_control_rl_server\examples\exp_1g_EXAMPLE\save_data'
hdf5_name = time.strftime('50pts-%Y%m%d-%H%M%S.h5')
epochs = 1000000
epochs_per_seg = 50
train_batch_size = 20
qubit_amp_scale = 4
cavity_amp_scale = 1
# ========== OPTIONS ========== #

qscale = []
cscale = []
for i in range(2 * start_split_num):
    qscale.append(init_qamp * qubit_amp_scale)
    cscale.append(init_camp * cavity_amp_scale)

drive_freqs = np.array(init_freqs)

# Setup initial times and pulse amplitudes
times = np.linspace(time_start, time_stop, start_split_num + 2)[1:-1]

t_segs = np.array([times for i in range(2 * num_drives)])

amp_segs = (np.random.rand(2 * num_drives, start_split_num) - 0.5)
amp_segs[:2, :] *= init_qamp
amp_segs[2:, :] *= init_camp

# Create blank history arrays for storing optimal / past values
time_hist = []
amp_hist = []
cost_hist = []

In [ ]:
# Run vqe, etc
vmax = np.vectorize(max)
vmin = np.vectorize(min)

hdf5_start_index = 0
start_segs = start_split_num
for i in range(max_segs):
    
    temp_amp_scale = np.append(np.array(qscale), np.array(cscale))
    contrast = 1.9485
    client_args = [chs, verbose, plot_opt_pulses, plot_pulse_viewer, time_start, time_stop, drive_freqs, t_segs.flatten()]
    server_args = [epochs, train_batch_size, amp_segs, temp_amp_scale, hdf5_name]

    # Save args for rl client
    cf_name = "temp_files/client_args.txt"
    with open(cf_name, "wb") as fp:
        pickle.dump(client_args, fp)
    fp.close()

    # Save args for rl server
    sf_name = "temp_files/server_args.txt"
    with open(sf_name, "wb") as fp:
        pickle.dump(server_args, fp)
    fp.close()


    os.system('cmd /c python ./run_rl_scripts.py')

    # Give time for files to be updated etc
    time.sleep(1)

    time.sleep(10)
    opt_amps = []
    opt_times = []
    opt_freqs = []

    with h5py.File(os.path.join(save_dir, hdf5_name), "r") as f:
        opt_res_index = np.argmax(f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()])
        opt_result = f[str(i + hdf5_start_index)]["evaluation"]["rewards"][()][opt_res_index]
        for j in range(2 * num_drives):
            opt_amps.append([f[str(i + hdf5_start_index)]["evaluation"]["actions"][f'pulse_array_{j}'][()][opt_res_index]])
        
    # updates amplitudes and frequencies with optimized values and reshape
    amp_segs = np.array(opt_amps)
    amp_segs = np.reshape(amp_segs, (num_drives * 2, int(len(amp_segs.flatten()) / (num_drives * 2))))

    print(f'================')
    print(f'num segs: {i + start_segs} ')
    print(f'opt_amps: {amp_segs}')
    print(f'opt_result: {opt_result}')

    # save values to history arrays
    time_hist.append(interp_time_wrapper(t_segs, time_start, time_stop))
    amp_hist.append(interp_amp_wrapper(amp_segs))
    cost_hist.append(opt_result)

    np.savez(r'run_data\\' + hdf5_name[:-3] + "-" + str(i) + ".npz", time=time_hist[-1], amp=amp_hist[-1], cost=cost_hist[-1])

    for i in range(2 * n_seg_jump):
        qscale.append(init_qamp * qubit_amp_scale)
        cscale.append(init_camp * cavity_amp_scale)

    # split segments and return to start of loop
    if (i < max_segs - 1):
        for i in range(n_seg_jump):
            t_segs, amp_segs = split_segs_flat(interp_time_wrapper(t_segs, time_start, time_stop), interp_amp_wrapper(amp_segs))